# Homework 3 -- Cloropleth map of grants in Swiss universities

In [145]:
# imports and all that
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os.path

### Data wrangling

In [146]:
grants = pd.read_csv("data/P3_GrantExport.csv", sep=';', index_col=0)
grants.size
grants.shape

(48464, 8)

We'll try to reduce the dataset by removing the entries that are not in CH and irrelevant columns.
We also remove data where location cannot be inferred, i.e. which are missing both the institute and the university

These changes can be safely persisted in the file, as it will not influence the data we work with.

In [147]:
grants = grants[grants.University.notnull()]
grants = grants.drop(axis=1, errors='ignore', labels=['Project Title', 'Project Title English',
            'Responsible Applicant', 'Funding Instrument', 'Funding Instrument Hierarchy', 'Keywords'])

# drop rows where location cannot be inferred
filter_no_location = ~((grants.University == "Nicht zuteilbar - NA") & (grants.Institution.isnull()))
grants = grants[filter_no_location]
grants.dtypes
grants.shape

(48464, 8)

In [148]:
# Sum entries in the Approved Amount column are not numeric and such are being dropped
grants['Approved Amount'] = pd.to_numeric(grants['Approved Amount'], errors='coerce')
grants = grants[grants['Approved Amount'].isnull() == False]
grants.shape

(48464, 8)

In [149]:
# Run if you want to update the csv
#grants.to_csv("data/P3_GrantExport.csv", sep=';') # keep the same sep

In [150]:
import re
def getLocationTagFromUni(s):
    return re.findall(r'\S+', s)[-1]

In [151]:
grants['Location Tag'] = grants['University'].apply(getLocationTagFromUni)
grants.head()

,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Location Tag
"﻿""Project Number""",,,,,,,,,
4,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.0,GE
5,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",10101,Philosophy,Human and Social Sciences;Linguistics and lite...,01.03.1976,28.02.1985,79732.0,NPO
6,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,52627.0,BS
7,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",10303,Ancient history and Classical studies,Human and Social Sciences;Theology & religious...,01.01.1976,30.04.1978,120042.0,NPO
8,"Séminaire de politique économique, d'économie ...",Université de Fribourg - FR,10203,Economics,"Human and Social Sciences;Economics, law",01.01.1976,31.12.1978,53009.0,FR


In [152]:
# Grouped by 'University' and the total funds each gets 
grants_grouped_by_uni = grants.groupby(['Location Tag', 'University'])
grants_grouped_by_uni = grants_grouped_by_uni.agg({'Approved Amount':'sum'}).reset_index()
grants_grouped_by_uni.shape

(77, 3)

Command line 'voodoo' magic to get the list of cantons from the json file 

In [153]:
cantons =! cat "ch-cantons.topojson.json" | grep "id\":" | cut -d\" -f4

In [154]:
len(cantons)

2

### Find location of universities

Use Google API to find the location corresponding to the university. This will only run once and save the results of the requests into a file.

In [155]:
google_key="0" #todo insert google api key
data = 'data/requests.txt'
url = "https://maps.googleapis.com/maps/api/place/textsearch/xml?key="+google_key+"&query="

def get_city(row):
    r = requests.get(url+row.University)
    with open(data, "a") as myfile:
        myfile.write(r.text)
                
if not os.path.isfile(data):
    with open(data, "a") as myfile:
        myfile.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?><results>")
    grants_grouped_by_uni.apply(get_city, axis=1)
    with open(data, "a") as myfile:
        myfile.write("</results>")

Load the results for the google requests and filter out the postalcodes of the Universities location. This will later be used to find the canton.

In [156]:
results = BeautifulSoup(open(data), "xml")
frame = grants_grouped_by_uni.copy()
frame['postalcode'] = 'None'
i = 0
for response in results.find_all('PlaceSearchResponse'):
    result = response.find('result')
    if result != None:
        city = result.find('formatted_address').text
        if "Switzerland" in city:
            pc = str(city[city.index(',')+2:city.index(',')+6])
            if (pc.isdigit()):
                frame.set_value(i, 'postalcode', pc)
            else:
                frame.set_value(i, 'postalcode', city)
    i += 1
frame.head(10)

,Location Tag,University,Approved Amount,postalcode
0,AGS,Forschungsanstalten Agroscope - AGS,3.311572e+07,None
1,AORI,AO Research Institute - AORI,3.435621e+06,None
2,ASP,Pädag. Hochschule Tessin (Teilschule SUPSI) - ASP,1.593170e+05,None
3,ASPIT,Weitere Spitäler - ASPIT,1.074981e+07,None
4,BE,Universität Bern - BE,1.519373e+09,3012
5,BFH,Berner Fachhochschule - BFH,3.102870e+07,3401
6,BITG,Biotechnologie Institut Thurgau - BITG,2.492535e+06,8280
7,BS,Universität Basel - BS,1.352251e+09,4003
8,CREALP,Centre de rech. sur l'environnement alpin - CR...,1.567678e+06,1950
9,CSEM,Swiss Center for Electronics and Microtech. - ...,1.806825e+07,None


Show the universities that where not found

In [157]:
print(frame[frame.postalcode=='None'].shape)
frame[frame.postalcode=='None']

(20, 4)


,Location Tag,University,Approved Amount,postalcode
0,AGS,Forschungsanstalten Agroscope - AGS,3.311572e+07,None
1,AORI,AO Research Institute - AORI,3.435621e+06,None
2,ASP,Pädag. Hochschule Tessin (Teilschule SUPSI) - ASP,1.593170e+05,None
3,ASPIT,Weitere Spitäler - ASPIT,1.074981e+07,None
9,CSEM,Swiss Center for Electronics and Microtech. - ...,1.806825e+07,None
20,FIBL,Forschungsinstitut für biologischen Landbau - ...,7.442410e+06,None
21,FINST,Weitere Institute - FINST,9.256736e+06,None
23,FORS,Schweizer Kompetenzzentrum Sozialwissensch. - ...,3.473582e+07,None
24,FP,Firmen/Privatwirtschaft - FP,1.116867e+08,None
38,IRO,Forschungsinstitut für Opthalmologie - IRO,3.478469e+06,None


For the universities that where found we will use the geonames api to find the canton based on the postalcode.

In [158]:
username = "juta" #todo add username
url = "http://api.geonames.org/postalCodeSearch?maxRows=1&username="+username+"&country=ch&postalcode="

def get_canton(row):
    if row.postalcode.isdigit():
        r = requests.get(url+row.postalcode)
        soup = BeautifulSoup(r.text, "xml")
        if soup.find('adminCode1'):
            return soup.find('adminCode1').text
        else:
            return 'None'
    else:
        return 'None'

frame['canton'] = frame.apply(get_canton, axis=1)
frame.head()

,Location Tag,University,Approved Amount,postalcode,canton
0,AGS,Forschungsanstalten Agroscope - AGS,3.311572e+07,None,None
1,AORI,AO Research Institute - AORI,3.435621e+06,None,None
2,ASP,Pädag. Hochschule Tessin (Teilschule SUPSI) - ASP,1.593170e+05,None,None
3,ASPIT,Weitere Spitäler - ASPIT,1.074981e+07,None,None
4,BE,Universität Bern - BE,1.519373e+09,3012,BE


In [159]:
df = frame.groupby(['canton'])
df = df.agg({'Approved Amount':'sum'}).reset_index()
df = df[df.canton != 'None']
df.to_csv("data/grants_per_canton.csv", sep=';')
df

,canton,Approved Amount
0,BE,1.555048e+09
1,BL,3.476142e+06
2,BS,1.392481e+09
3,FR,4.590737e+08
4,GE,1.158322e+07
5,GR,2.100426e+07
6,JU,3.479035e+07
7,LU,5.467329e+07
8,NE,3.832046e+08
10,SG,8.675705e+07


TODO:

* we got 77 location tags and 26 cantons
* so the non-canton locations need to be mapped to cantons
* identify the 'how'
* also split NPO